##Hei! Her kan du se en rekke eksempler på bruk av tabellen `nvdb_vegpunkter` i praksis.<br>Merk at vegpunkter tabellen alltid kun holder inneværende mnd,<br> og historikken ligger i "_history".<br><br>NB! I noen tilfeller har vi brukt _history varianten av tabellen, i de eksemplene der vi viser en spesifikk punkt_id. Dette for at eksempelet ikke skal slutte å virke en gang i fremtiden.

In [ ]:
project = 'saga-data'
use_colab_auth = True

# Legg inn ditt eget prosjekt her, f.eks. 'saga-olanor-playground-ab12'
bq_job_project = ''

In [ ]:
if (use_colab_auth):
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')

In [ ]:
import warnings
from google.cloud import bigquery

warnings.filterwarnings('ignore')
client = bigquery.Client(project=bq_job_project)

## Eksempel 1

Jeg har noen veibredde-data som jeg gjerne skulle ha beriket med fartsgrense.

In [ ]:
query = f"""
-- Vi lager en rad for å simulere dataene.
with veibredde as (
  select 72611 as veglenkesekvensid, 0.578742 as startposisjon, 1.0 as sluttposisjon, 6 as veibredde
)

-- Vi må koble dette til vegpunktene for å enkelt kunne koble på fartsgrenser.
-- Først henter vi vegpunktene
, vegpunkter as (
  select * from saga-data.nvdb.nvdb_vegpunkter
)

-- Og kobler veibreddedataene til vegpunktene for å få informasjonen på metersnivå
, joined_step_1 as (
  select
    a.punkt_id
  , a.veglenkesekvensid
  , a.relativ_posisjon
  , b.veibredde
  from vegpunkter as a
  inner join veibredde as b
    on a.veglenkesekvensid = b.veglenkesekvensid
    and a.relativ_posisjon between b.startposisjon and b.sluttposisjon
)

-- Vi henter så fartsgrenser
, fartsgrenser as (
  select
    egenskaper.fartsgrenseverdi as fartsgrense
  , ls.veglenkesekvensid
  , ls.startposisjon
  , ls.sluttposisjon
  from saga-nvdb-prod-vlmh.standardized.vegobjekter_fartsgrense,
  unnest(lokasjon.stedfestinger) as ls
  where (metadata.sluttdato is null or metadata.sluttdato >= current_date())
)

-- Vi kobler fartsgrenser på vegpunktene
, joined_all as (
  select
    a.*
  , b.fartsgrense
  from joined_step_1 as a
  inner join fartsgrenser as b
    on a.veglenkesekvensid = b.veglenkesekvensid
    and a.relativ_posisjon between b.startposisjon and b.sluttposisjon
)

-- Og ruller det opp igjen til det originale formatet
  select
    veglenkesekvensid
  , veibredde
  , fartsgrense
  , min(relativ_posisjon) as startposisjon
  , max(relativ_posisjon) as sluttposisjon
  from joined_all
  group by 1,2,3
  ;
"""

client.query(query).to_dataframe()

## Eksempel 2
Jeg har fått noen data med såkalt "lokasjon.stedfestinger.kortform".
For eksempel "0.01587524-0.2725013@971774". Hvor i Norge er dette?

In [ ]:
query = f"""
-- Vi lager en rad for å simulere dataene.
with eksempeldata as (
  select "0.01587524-0.2725013@971774" as kortform, 42 as measurement
)

-- Dette er egentlig veglenkesekvens id og relative posisjoner, så dataene må brytes opp litt først.
, eksempeldata_brutt_opp_steg1 as (
  select
    split(kortform, '@')[OFFSET(0)] as start_til_sluttposisjon
  , split(kortform, '@')[OFFSET(1)] as veglenkesekvensid
  from eksempeldata
)

-- Deretter kaller vi variablene det som dataene representerer
, eksempeldata_brutt_opp_steg2 as (
  select
    cast(veglenkesekvensid as int64) as veglenkesekvensid
  , cast(SPLIT(start_til_sluttposisjon, '-')[OFFSET(0)] as float64) as startposisjon
  , cast(SPLIT(start_til_sluttposisjon, '-')[OFFSET(1)] as float64) as sluttposisjon
  from eksempeldata_brutt_opp_steg1
)

-- Henter inn vegpunktene
, vegpunkter as (
  select * from saga-data.nvdb.nvdb_vegpunkter
)

-- Og kobler det hele sammen
, joined_to_vegpunkter as (
  select
    vegkategori
  , vegnummer
  , kommunenummer
  , sum(avstand_til_neste_punkt) as ca_antall_meter
  from eksempeldata_brutt_opp_steg2 as a
  inner join vegpunkter as b
    on a.veglenkesekvensid = b.veglenkesekvensid
    and b.relativ_posisjon between a.startposisjon and a.sluttposisjon
  group by 1,2,3
  order by 1,2,3
)

-- Så lurer vi på hvilken kommune som kommunenummeret tilhører - vi henter det inn
, join_kommunenavn as (
  select
    a.*, b.kommunenavn
  from joined_to_vegpunkter as a
  left join saga-data.geografi.kommuner_2020_v2 as b
    on a.kommunenummer = b.kommunenummer
)

select * from join_kommunenavn
"""

client.query(query).to_dataframe()

Svar: Stedfestingen er på E6 i Halden, og er ca 1.66 km lang.

## Eksempel 3
Jeg har to datasett som kun er geografiske objekter (linestrings) men de har ikke noe mer info. Hvordan kan jeg koble datasettene sammen der geografien overlapper, og berike dataene med informasjon om vegnettet fra NVDB Vegpunkter)?

In [ ]:
query = f"""
-- Vi lager først begge eksempeldatasettene
with data1 as (
  select
    172 as id
  , ST_GeogFromText('LINESTRING(10.227466 59.723153, 10.22756 59.723072, 10.22765 59.722985\
  , 10.227688 59.722949, 10.227723 59.722912, 10.227759 59.722874, 10.227794 59.722838, 10.227828 59.7228\
  , 10.227862 59.722763, 10.227896 59.722725, 10.227928 59.722688, 10.22796 59.722649, 10.22799 59.722612)') as geometri
  , 42 as measurement_one
)

, data2 as (
  select
    172 as id
  , ST_GeogFromText('LINESTRING(10.227688 59.722949, 10.227723 59.722912, 10.227759 59.722874, 10.227794 59.722838)') as geometri
  , 78 as measurement_two
)

-- Vi beriker datasettene med kvadratiske grids for raskere påkobling mot NVDB Vegpunkter
, add_squaregrids_to_data1 as (
  select
    a.*
  , d.square_gridcell_id_small
  from
    data1 as a
  inner join saga-data.geografi.norway_square_polygons_large as b
    on st_intersects(a.geometri, b.geometri)
  inner join saga-data.geografi.norway_square_polygons_medium as c
    on b.x_index_large = c.x_index_large
    and b.y_index_large = c.y_index_large
    and st_intersects(a.geometri, c.geometri)
  inner join saga-data.geografi.norway_square_polygons_small as d
    on c.x_index_medium = d.x_index_medium
    and c.y_index_medium = d.y_index_medium
    and st_intersects(a.geometri, d.geometri)
)

, add_squaregrids_to_data2 as (
  select
    a.*
  , d.square_gridcell_id_small
  from
    data2 as a
  inner join saga-data.geografi.norway_square_polygons_large as b
    on st_intersects(a.geometri, b.geometri)
  inner join saga-data.geografi.norway_square_polygons_medium as c
    on b.x_index_large = c.x_index_large
    and b.y_index_large = c.y_index_large
    and st_intersects(a.geometri, c.geometri)
  inner join saga-data.geografi.norway_square_polygons_small as d
    on c.x_index_medium = d.x_index_medium
    and c.y_index_medium = d.y_index_medium
    and st_intersects(a.geometri, d.geometri)
)

-- Vi henter vegpunktene
, vegpunkter as (
  select * from saga-data.nvdb.nvdb_vegpunkter
)

select
  a.*, b.measurement_one, c.measurement_two
from vegpunkter as a
inner join add_squaregrids_to_data1 as b
  on a.square_gridcell_id_small = b.square_gridcell_id_small
  and st_intersects(st_buffer(b.geometri,10), a.punkt_geometri)
inner join add_squaregrids_to_data2 as c
  on a.square_gridcell_id_small = c.square_gridcell_id_small
  and st_intersects(st_buffer(c.geometri,10), a.punkt_geometri)
order by veglenkesekvensid, relativ_posisjon asc
"""

#print(query)

client.query(query).to_dataframe()

Vi kobler datasettene der geografien overlapper, og vi får ut masse nyttig info om punktene de treffer på vegnettet.

## Eksempel 4
Jeg vil ha geografiske linestrings for E6, omtrent mellom Trondheim og Oppdal (sør-/nordående).

In [ ]:
query = f"""
with vegpunkter as (
  select * from saga-data.nvdb.nvdb_vegpunkter
  where vegkategori = "E"
  and vegnummer = 6
  and vsr_objekt = "normal_strekning"
)

, koble_paa_kommunenavn as (
  select
    a.*, b.kommunenavn
  from vegpunkter as a
  left join saga-data.geografi.kommuner_2020_v2 as b
    on a.kommunenummer = b.kommunenummer
)

, filtrer_kommune as (
  select
    veglenkesekvensid
  , veglenke_referanse
  , vl_punktnummer
  , vl_subpunktnummer
  , punkt_geometri
  from koble_paa_kommunenavn
  -- Vi henter bare vegpunkter som ligger mellom disse to kommunene
  -- NB. Bruk av grader_nord eller grader_ost avhenger av plasseringen mellom lokasjonene du er interessert i
  -- For Oslo-Bergen ville det vært naturlig å bruke grader_ost
  where grader_nord > (select avg(grader_nord) from koble_paa_kommunenavn where kommunenavn = "Oppdal")
  and grader_nord < (select avg(grader_nord) from koble_paa_kommunenavn where kommunenavn = "Trondheim")
  order by 1,2,3,4 asc
)

select
  veglenkesekvensid
, veglenke_referanse
, st_makeline(array_agg(punkt_geometri)) as linestrings
from filtrer_kommune
group by 1,2
"""

#print(query)

client.query(query).to_dataframe()

Tips: Se på linestringene i GeoViz for å se om de virker OK før du bruker de.

## Eksempel 5
Hvilken kommune har flest vegpunkter med sykkelfelt?

In [ ]:
query = f"""
with vegpunkter as (
  select * from saga-data.nvdb.nvdb_vegpunkter
  where vegkategori in ("E", "R")
  and vl_type_veg = "Enkel bilveg"
)

, koble_paa_kommunenavn as (
  select
    a.*, b.kommunenavn
  from vegpunkter as a
  left join saga-data.geografi.kommuner_2020_v2 as b
    on a.kommunenummer = b.kommunenummer
)

select
  kommunenavn,
  sum(case when vl_har_sykkelfelt then 1 else 0 end) / sum(1) as andel_sykkelfelt
from koble_paa_kommunenavn
group by kommunenavn
order by andel_sykkelfelt desc
"""

#print(query)

client.query(query).to_dataframe()

Vi får sortert på kommuner med størst relativ andel vegpunkter for sykkelfelt øverst.

## Eksempel 6
Sjekk andelen veglenkesekvenser på Europavei fra Nordland og nordover som har en rasteplass eller dægnhvilkeplass et sted på lenkesekvensen.

In [ ]:
query = f"""
with rasteplass as (
  select
    1 as rasteplass,
    lokasjon.kommuner,
    ls.veglenkesekvensid,
    ls.startposisjon,
    ls.sluttposisjon
  from `saga-nvdb-prod-vlmh.standardized.vegobjekter_rasteplass`
  cross join unnest(lokasjon.stedfestinger) as ls
  where metadata.sluttdato is null
)

, dognhvileplass as (
  select
    1 as dognhvileplass,
    lokasjon.kommuner,
    ls.veglenkesekvensid,
    ls.startposisjon,
    ls.sluttposisjon
  from `saga-nvdb-prod-vlmh.standardized.vegobjekter_dognhvileplass`
  cross join unnest(lokasjon.stedfestinger) as ls
  where metadata.sluttdato is null
)

, vegpunkter as (
  select *
  from saga-data.nvdb.nvdb_vegpunkter
  where vegkategori = "E"
  and grader_nord > (select min(grader_nord) from saga-data.nvdb.nvdb_vegpunkter where fylkesnummer = 18)
)

, joined as (
  select
    a.*,
    coalesce(b.rasteplass, 0) as rasteplass, coalesce(c.dognhvileplass, 0) as dognhvileplass
  from vegpunkter  as a
  left join rasteplass as b
    on a.veglenkesekvensid = b.veglenkesekvensid
    and a.relativ_posisjon between b.startposisjon and b.sluttposisjon
  left join dognhvileplass as c
    on a.veglenkesekvensid = c.veglenkesekvensid
    and a.relativ_posisjon between c.startposisjon and c.sluttposisjon
)

, har_eller_har_ikke as (
  select
    veglenkesekvensid
  , max(case when (rasteplass + dognhvileplass) > 0 then 1 else 0 end) as raste_eller_dognhvileplass
  from joined
  group by 1
)

select avg(b.raste_eller_dognhvileplass) as andel_vls
from joined as a
inner join har_eller_har_ikke as b
  on a.veglenkesekvensid = b.veglenkesekvensid
"""

#print(query)

client.query(query).to_dataframe()

Vi ser hvilken andel veglenkesekvenser som er tilknyttet raste eller døgnhvileplass

## Eksempel 7
Hvilken kommune og veitype hadde flest registrerte asfaltarbeid i 2022?

In [ ]:
query = f"""
-- NB: Logikken vil dobbeltelle i de tilfellene der samme asfaltarbeid traff flere kommuner.
with vegpunkter as (
  select * from saga-data.nvdb.nvdb_vegpunkter
  where vegkategori in ("E", "R")
)

, paved_road as (
  select id as asfalt_id
  , ls.veglenkesekvensid
  , ls.startposisjon
  , ls.sluttposisjon
  from `saga-nvdb-prod-vlmh.standardized.vegobjekter_vegdekke`, unnest(lokasjon.stedfestinger) ls
  where extract(year from egenskaper.dekkeleggingsdato) = 2022
  and metadata.sluttdato is null
)

, joined as (
  select a.*, b.asfalt_id
  from vegpunkter as a
  left join paved_road as b
    on a.veglenkesekvensid = b.veglenkesekvensid
    and a.relativ_posisjon between b.startposisjon and b.sluttposisjon
)

, koble_paa_kommunenavn as (
  select
    a.*, b.kommunenavn
  from joined as a
  left join saga-data.geografi.kommuner_2020_v2 as b
    on a.kommunenummer = b.kommunenummer
)

select
  kommunenavn,
  vegkategori,
  count(distinct asfalt_id) as antall_asfalteringer
from koble_paa_kommunenavn
group by 1,2
order by 3 desc
"""

#print(query)

client.query(query).to_dataframe()

Vi ser antall asfalteringer sortert på høyest øverst.

## Eksempel 8
Jeg har fått en måling med vegsystemreferanse med bokstaver og meter.<br>
Kan jeg få et datasett om denne strekningen?

**NB:** vegsystemreferanse kortform er bare gyldige frem til det skjer en endring på for vegsystemreferansen - f.eks. kan metreringen oppdaters. Husk å koble på nærmeste run_yearmonth fra nvdb_vegpunkter_history

In [ ]:
query = f"""
with eksempeldata as (
  select
    "EV6 S9D1 m0-70" as kortform,
    date("2023-10-05") as measurement_date,
    42 as measurement
)

-- Vegsystemreferansen må brytes opp på denne måten for å kobles til NVDB Vegpunkter
, eksempeldata_brutt_opp as (
  select
    trim(regexp_replace(kortform, regexp_extract(kortform, r"m\d+-\d+"), "")) as vl_vegsystemref_rot
  , cast(replace(split(regexp_extract(kortform, r"m\d+-\d+"), "-")[safe_offset(0)], "m", "") as int64) as vl_vegsystemref_fra_meter
  , cast(split(regexp_extract(kortform, r"m\d+-\d+"), "-")[safe_offset(1)] as int64) as vl_vegsystemref_til_meter
  , measurement
  from eksempeldata
)

, vegpunkter as (
  select *
  from saga-data.nvdb.nvdb_vegpunkter_history
  where run_yearmonth = (
    select cast(cast(extract(year from measurement_date) as string) || format("%02d", extract(month from measurement_date)) as int64) from eksempeldata
  )
)

select
  b.*,
  a.measurement
from eksempeldata_brutt_opp as a
inner join vegpunkter as b
  on a.vl_vegsystemref_rot = b.vl_vegsystemref_rot
  and b.metrering between a.vl_vegsystemref_fra_meter and a.vl_vegsystemref_til_meter
"""

#print(query)

client.query(query).to_dataframe()

Vipps, et datasett med mye mer info.



## Eksempel 9
I NVDB Vegpunker finnes tilknytning mellom unike vegpunkter og ruter i det minste kvadratiske rutenettet.<br>
Hvordan kan jeg gå fra små ruter til større ruter i de kvadratiske rutenettene?<br>
Spørringen under kan left joines mot NVDB Vegpunkter (square_gridcell_id_small).

In [ ]:
query = """
select
  s.square_gridcell_id_small,
  m.square_gridcell_id_medium,
  l.square_gridcell_id_large,
  s.geometri as squaregrid_geo_small,
  m.geometri as squaregrid_geo_medium,
  l.geometri as squaregrid_geo_large
from
  saga-data.geografi.norway_square_polygons_small s
left join saga-data.geografi.norway_square_polygons_medium m
  on s.x_index_medium = m.x_index_medium
  and s.y_index_medium = m.y_index_medium
left join saga-data.geografi.norway_square_polygons_large l
  on m.x_index_large = l.x_index_large
  and m.y_index_large = l.y_index_large
limit 20
;
"""

client.query(query).to_dataframe()

## Eksempel 10
Jeg har en punkt_id i NVDB Vegpunkter<br>
Hvordan jeg kan gå fra punktets rute (i det minste kvadratiske rutenettet) til en større kvadratisk rute?<br>
I eksempelet er punkt_id'en gjeldende i run_yearmonth 202310 i NVDB Vegpunkter.

In [ ]:
query = """
with punkt as (
  select punkt_id, punkt_geometri, run_yearmonth
  from saga-data.nvdb.nvdb_vegpunkter_history
  where run_yearmonth = 202310 
  and punkt_id = "625212-24-14-12-5-u4xsw6vq9sk77h2mcgjc"
)
, grid_mapping as (
select
  a.punkt_id,
  b.punkt_geometri,
  a.* except (punkt_id)
from
  saga-data-stm.geografi.norway_square_polygons_to_nvdb_vegpunkter_mapping_history a
inner join punkt b
  on a.punkt_id = b.punkt_id
  and a.run_yearmonth = b.run_yearmonth
)
select
  a.*,
  b.geometri
from grid_mapping a
inner join saga-data.geografi.norway_square_polygons_small b
  on a.square_gridcell_id_small = b.square_gridcell_id_small
/*
inner join saga-data.geografi.norway_square_polygons_medium c
  on a.square_gridcell_id_medium = c.square_gridcell_id_medium
inner join saga-data.geografi.norway_square_polygons_large d
  on a.square_gridcell_id_large = d.square_gridcell_id_large
*/
;
"""

client.query(query).to_dataframe()

## Eksempel 11
Heksagonale rutenett er hierarkiske, dvs. at enhver rute har en forelder-rute.<br>
Obs! Om du skal matche et unikt punkt mot ruter av forskjellige størrelser, bør du følge eksempelet under (Eksempel 12). Dette er fordi et punkt ikke nødvendig er innenfor forelder-rute (Y) til rute X, selv om hierarkiet for heksagonale ruter tilsier at rute X har forelder Y.<br><br>
Følgende SQL kan brukes som utgangspunkt for å visualisere ruter av forskjellige størrelser.

In [ ]:
query = """
with hexagons as (
  select
    r8.h3_index_res_8,
    r7.h3_index_res_7,
    r6.h3_index_res_6,
    r5.h3_index_res_5,
    r8.geometri as hexgrid_geo_r8,
    r7.geometri as hexgrid_geo_r7,
    r6.geometri as hexgrid_geo_r6,
    r5.geometri as hexgrid_geo_r5
  from
    saga-data.geografi.norway_hexagon_polygons_resolution_8 r8
  left join saga-data.geografi.norway_hexagon_polygons_resolution_7 r7
    on r8.parent_h3_index = r7.h3_index_res_7
  left join saga-data.geografi.norway_hexagon_polygons_resolution_6 r6
    on r7.parent_h3_index = r6.h3_index_res_6
  left join saga-data.geografi.norway_hexagon_polygons_resolution_5 r5
    on r6.parent_h3_index = r5.h3_index_res_5
)
select *
from hexagons
/*
-- Enkelte ruter har ikke en forelder.
-- Disse ligger likevel utenfor Norge, så det er ikke et problem.
where (h3_index_res_7 is null
  or h3_index_res_6 is null
  or h3_index_res_5 is null)
*/
;
"""

client.query(query).to_dataframe()

## Eksempel 12
Jeg har en punkt_id i NVDB Vegpunkter.<br>
Hvordan jeg kan gå fra punktets rute (i det minste heksagonale rutenettet) til en større heksagonal rute?<br>
I eksempelet er punkt_id'en gjeldende i run_yearmonth 202310 i NVDB Vegpunkter.

In [ ]:
query = """
with punkt as (
  select punkt_id, punkt_geometri, run_yearmonth
  from saga-data.nvdb.nvdb_vegpunkter_history
  where run_yearmonth = 202310 
  and punkt_id = "625212-24-14-12-5-u4xsw6vq9sk77h2mcgjc"

  
)
, grid_mapping as (
select
  a.punkt_id,
  b.punkt_geometri,
  a.* except (punkt_id)
from
  saga-data-stm.geografi.norway_hexagon_polygons_to_nvdb_vegpunkter_mapping_history a
inner join punkt b
  on a.punkt_id = b.punkt_id
  and a.run_yearmonth = b.run_yearmonth
)
select
  a.*,
  b.geometri
from grid_mapping a
inner join saga-data.geografi.norway_hexagon_polygons_resolution_8 b
  on a.h3_index_res_8 = b.h3_index_res_8
/*
inner join saga-data.geografi.norway_hexagon_polygons_resolution_7 c
  on a.h3_index_res_7 = c.h3_index_res_7
inner join saga-data.geografi.norway_hexagon_polygons_resolution_6 d
  on a.h3_index_res_6 = d.h3_index_res_6
inner join saga-data.geografi.norway_hexagon_polygons_resolution_5 e
  on a.h3_index_res_5 = e.h3_index_res_5
*/
;
"""

client.query(query).to_dataframe()